<a href="https://colab.research.google.com/github/yakuzadave/JupyterDungeonMaster/blob/main/LangChain_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain: Tools & Chains

In [39]:
!pip -q install tiktoken pinecone-client pinecone-text weaviate-client openai langchain huggingface_hub d20 pinecone-client google-search-results gspread PyYAML python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.6 MB/s eta 0:00:00


In [ ]:
!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
!mv openapi.yaml openai_openapi.yaml
!wget https://www.klarna.com/us/shopping/public/openai/v0/api-docs
!mv api-docs klarna_openapi.yaml
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

In [17]:
!git clone https://github.com/hwchase17/langchain-hub

Cloning into 'langchain-hub'...
remote: Enumerating objects: 455, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 455 (delta 28), reused 32 (delta 17), pack-reused 390
Receiving objects: 100% (455/455), 75.37 KiB | 2.79 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [25]:
import os
from dotenv import load_dotenv
import pinecone


# Add Vars here 

api_key = os.getenv("PINECONE_API_KEY")
# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

index_name = "langchain-pinecone-hybrid-search"

pinecone.init(api_key=api_key, enviroment=env)
pinecone.whoami()

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


WhoAmIResponse(username='9eb4a41', user_label='default', projectname='1780fef')

# Pinecone

## Index Setup

In [26]:
 # create the index
pinecone.create_index(
   name = index_name,
   dimension = 1536,  # dimensionality of dense model
   metric = "dotproduct",  # sparse values supported only for dotproduct
   pod_type = "s1",
   metadata_config={"indexed": []}  # see explaination above
)

In [27]:
index = pinecone.Index(index_name)

### For Sparse Embeddings

In [31]:
from pinecone_text.sparse import BM25Encoder
# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
corpus = ["foo", "bar", "world", "hello"]

# fit tf-idf values on your corpus
bm25_encoder.fit(corpus)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/4 [00:00<?, ?it/s]

NameError: ignored

# Python Imports

In [7]:
from langchain.chains import PALChain, OpenAPIEndpointChain
from langchain.prompts import PromptTemplate, Prompt,AIMessagePromptTemplate
from langchain.llms import OpenAI, LlamaCpp, HuggingFaceHub, HuggingFaceEndpoint, OpenAIChat
from langchain.chains import LLMChain, TransformChain, AnalyzeDocumentChain, APIChain
from langchain import OpenAI, LLMChain
from langchain.retrievers import PineconeHybridSearchRetriever

## Models

In [8]:
from langchain.chat_models import ChatOpenAI
chat_llm = ChatOpenAI(model_name='gpt-3.5-turbo', 
             temperature=0, 
             max_tokens=1024)

In [9]:
oai_llm = OpenAI(model_name='text-davinci-003', 
             temperature=0, 
             max_tokens = 256)

## Chains

In [10]:
from langchain.chains import SimpleSequentialChain, SequentialChain, APIChain
from langchain.chains.api import open_meteo_docs, podcast_docs

## Prompts

In [11]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.prompts.prompt import PromptTemplate

## Embeddings 

In [35]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [37]:
from langchain.retrievers import PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [40]:
retriever.add_texts(["foo", "bar", "world", "hello"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
result = retriever.get_relevant_documents("foo")

In [42]:
result[0]

Document(page_content='foo', metadata={})

## Memory

In [12]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory

### Chat Message History

In [43]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [44]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}),
 AIMessage(content='whats up?', additional_kwargs={})]

### ConversationBufferMemory

In [45]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [46]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

In [47]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [48]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}),
  AIMessage(content='whats up?', additional_kwargs={})]}

In [49]:
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [50]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [51]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

### Saving Message History

In [52]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [53]:
dicts = messages_to_dict(history.messages)

In [54]:
dicts

[{'type': 'human', 'data': {'content': 'hi!', 'additional_kwargs': {}}},
 {'type': 'ai', 'data': {'content': 'whats up?', 'additional_kwargs': {}}}]

In [55]:
new_messages = messages_from_dict(dicts)

In [56]:
new_messages

[HumanMessage(content='hi!', additional_kwargs={}),
 AIMessage(content='whats up?', additional_kwargs={})]

### Entity History

In [57]:
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

In [58]:
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Dave is working on a Grim Dark Future Game app"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"ouput": " Grim Dark Future is a tabletop game based on Warhammer 40K.  The rules are designed to be flexible and modular, so players can customize their games and add their own unique rules and scenarios. "}
)

In [59]:
memory.load_memory_variables({"input": 'What is Grim Dark Future?'})

{'history': 'Human: Dave is working on a Grim Dark Future Game app\nAI:  Grim Dark Future is a tabletop game based on Warhammer 40K.  The rules are designed to be flexible and modular, so players can customize their games and add their own unique rules and scenarios. ',
 'entities': {'Grim Dark Future': ''}}

#### Using in a Chain

In [60]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [62]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [63]:
conversation.predict(input="What are the rules of Grim Dark Future?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Grim Dark Future is a post-apocalyptic setting where the world has been ravaged by war and chaos. The rules of the game are designed to create a gritty and dangerous atmosphere, where players must fight for survival and resources. The game focuses on scavenging, crafting, and combat, with a variety of weapons and equipment available to players. Additionally, the game features a unique morality system, where players must make difficult decisions that can have lasting consequences.'

In [64]:
conversation.memory.entity_store.store

{'Dave': 'Dave is working on a Grim Dark Future Game app.',
 'Grim Dark Future Game': 'Grim Dark Future Game is a tabletop game based on Warhammer 40K, with customizable rules and scenarios, and Dave is currently working on an app for it.',
 'Grim Dark Future': 'Grim Dark Future is a post-apocalyptic setting where the world has been ravaged by war and chaos, featuring scavenging, crafting, and combat with a variety of weapons and equipment, as well as a unique morality system with lasting consequences.'}

In [65]:
conversation.predict(input="I am trying to build an army in Grim Dark Future.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Building an army in Grim Dark Future requires careful planning and strategy. You will need to consider the resources available to you, such as weapons, equipment, and supplies, as well as the type of army you want to build. You will also need to consider the terrain and environment you will be fighting in, as well as the type of enemies you will be facing. Finally, you will need to decide on a strategy for how to deploy your forces and how to best use them in battle.'

In [66]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Dave': 'Dave is working on a Grim Dark Future Game app.',
 'Grim Dark Future': 'Grim Dark Future is a post-apocalyptic setting where the '
                     'world has been ravaged by war and chaos, featuring '
                     'scavenging, crafting, and combat with a variety of '
                     'weapons and equipment, as well as a unique morality '
                     'system with lasting consequences. Building an army in '
                     'Grim Dark Future requires careful planning and strategy, '
                     'including consideration of resources, terrain, '
                     'environment, and enemies.',
 'Grim Dark Future Game': 'Grim Dark Future Game is a tabletop game based on '
                          'Warhammer 40K, with customizable rules and '
                          'scenarios, and Dave is currently working on an app '
                          'for it.'}


In [67]:
conversation.predict(input="What do you know about Grim Dark Future?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Grim Dark Future is a post-apocalyptic setting where the world has been ravaged by war and chaos. It features scavenging, crafting, and combat with a variety of weapons and equipment, as well as a unique morality system with lasting consequences. Building an army in Grim Dark Future requires careful planning and strategy, including consideration of resources, terrain, environment, and enemies. Additionally, the game features a variety of characters and factions, each with their own motivations and goals.'

## Agents

In [13]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

## Schema

In [14]:
from langchain.schema import messages_from_dict, messages_to_dict

## Tools

In [15]:
from langchain.utilities import GoogleSearchAPIWrapper

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("tabletop games"))

GameTopia


# LangChain Hub

In [19]:
from langchain.chains import load_chain

llm_requests = load_chain('/content/langchain-hub/chains/llm-requests/chain.json')
llm_math = load_chain('/content/langchain-hub/chains/llm-math/chain.json')
llm_bash = load_chain('/content/langchain-hub/chains/llm-bash/chain.json')

In [ ]:
from langchain.agents import initialize_agent

llm = ...
tools = ...

agent = initialize_agent(tools, llm, agent="/content/langchain-hub/agents/self-ask-with-search/agent.json")

## Basic LLMChain - Fact Extraction

In [7]:
# fact_extraction_prompt = PromptTemplate(
#     input_variables=["text_input"],
#     template="Extract the key facts out of this text. Don't include opinions. Give each fact a number and keep them short sentences. :\n\n {text_input}"
# )
# fact_extraction_chain = LLMChain(llm=llm, prompt=fact_extraction_prompt)

# facts = fact_extraction_chain.run(article)

# print(facts)


## Rewrite as a summary from the facts

In [10]:
template_content = """
You are a Goldman Sachs analyst.
Take the following list of facts and use them to write a short paragrah for investors.
 Don't leave out key info:
 
{facts}
"""
investor_update_prompt = PromptTemplate(
    input_variables=["facts"],
    template=template_content
)

In [11]:
# investor_update_chain = LLMChain(llm=llm, prompt=investor_update_prompt)

# investor_update = investor_update_chain.run(facts)

# print(investor_update)
# len(investor_update)

 the start of 2022.

Coinbase released its Q4 2022 earnings on Tuesday, reporting total revenue of $605 million, which was slightly above Wall Street's expectations of $581.2 million. Despite the positive revenue figure, Coinbase reported a GAAP loss of $557 million in the three-month period. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022, while institutional volumes across the same timeframe fell from $133 billion to $125 billion. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022, leading to a 50% and 66% year-over-year decline in Coinbase's total trading volumes and transaction revenues, respectively. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022, while its "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022. Despite the overall decline 

1105

In [12]:
# triples_prompt = PromptTemplate(
#     input_variables=["facts"],
#     template="Take the following list of facts and turn them into triples for a knowledge graph:\n\n {facts}"
# )

In [13]:
# triples_chain = LLMChain(llm=llm, prompt=triples_prompt)

# triples = triples_chain.run(facts)

# print(triples)
# len(triples)

 the start of 2022.

1. (Coinbase, released, Q4 2022 earnings)
2. (Coinbase, generated, $605 million total revenue)
3. (Coinbase, lost, $557 million GAAP basis)
4. (Wall Street, expected, $581.2 million revenue)
5. (Coinbase's stock, risen, 86% year-to-date)
6. (Consumer trading volumes, fell, $26 billion Q3 2022)
7. (Institutional volumes, fell, $133 billion)
8. (Overall crypto market capitalization, fell, $1.5 trillion)
9. (Coinbase's total trading volumes, fell, 50% year-over-year)
10. (Coinbase's transaction revenues, fell, 66% year-over-year)
11. (Trading revenue, fell, $365.9 million Q3 2022)
12. (Coinbase's subscription and services revenue, rose, $210.5 million Q3 2022)
13. (Monthly active developers in crypto, more than doubled, since start of 2022)


768

## Chaining these together

In [14]:


# full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, investor_update_chain], verbose=True)

In [15]:
# response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...

1. Coinbase released its Q4 2022 earnings on Tuesday.
2. Coinbase generated $605 million in total revenue in Q4 2022.
3. Coinbase lost $557 million in the three-month period on a GAAP basis.
4. Wall Street expected Coinbase to report $581.2 million in revenue.
5. Coinbase's stock had risen 86% year-to-date before its Q4 earnings were released.
6. Consumer trading volumes fell from $26 billion in Q3 2022 to $20 billion in Q4 2022.
7. Institutional volumes across the same timeframe fell from $133 billion to $125 billion.
8. The overall crypto market capitalization fell about 64%, or $1.5 trillion during 2022.
9. Coinbase's total trading volumes and transaction revenues fell 50% and 66% year-over-year, respectively.
10. Trading revenue at Coinbase fell from $365.9 million in Q3 2022 to $322.1 million in Q4 2022.
11. Coinbase's "subscription and services revenue" rose from $210.5 million in Q3 2022 to $282.8 million in Q4 2022.
12. Monthly a

## PAL Math Chain

In [54]:
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

In [55]:
question = "Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"

In [56]:
question_02= "The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more, how many apples do they have?"

In [57]:
pal_chain.run(question_02)



> Entering new PALChain chain...
def solution():
    """The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more, how many apples do they have?"""
    apples_initial = 23
    apples_used = 20
    apples_bought = 6
    apples_left = apples_initial - apples_used + apples_bought
    result = apples_left
    return result

> Finished chain.


'9'

## API Chains - OpenMeteo - Weather information

can throw errors based on API return length

In [59]:
llm = OpenAI(temperature=0,
             max_tokens=100)

In [60]:

chain_new = APIChain.from_llm_and_api_docs(llm, 
                                           open_meteo_docs.OPEN_METEO_DOCS, 
                                           verbose=True)

In [61]:
chain_new.run('What is the temperature like right now in Bedok, Singapore in degrees Celcius?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=1.3&longitude=103.9&hourly=temperature_2m&current_weather=true&temperature_unit=celsius
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.15091896057128906,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":6.0,"current_weather":{"temperature":30.7,"windspeed":8.4,"winddirection":205.0,"weathercode":3,"is_day":1,"time":"2023-04-21T05:00"},"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-04-21T00:00","2023-04-21T01:00","2023-04-21T02:00","2023-04-21T03:00","2023-04-21T04:00","2023-04-21T05:00","2023-04-21T06:00","2023-04-21T07:00","2023-04-21T08:00","2023-04-21T09:00","2023-04-21T10:00","2023-04-21T11:00","2023-04-21T12:00","2023-04-21T13:00","2023-04-21T14:00","2023-04-21T15:00","2023-04-21T16:00","2023-04-21T17:00","2023-04-21T18:00","2023-04-21T19:00","2023-04-21T20:00","2023-04-21T21:00","2023-04-21T22:00","2023-04-21T23:00","2023

' The temperature right now in Bedok, Singapore is 30.7 degrees Celcius.'

In [ ]:
chain_new.run('Is it raining in Singapore?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=1.3521&longitude=103.8198&current_weather=true&rain=true
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.41103363037109375,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":46.0,"current_weather":{"temperature":25.8,"windspeed":10.5,"winddirection":16.0,"weathercode":3,"time":"2023-02-22T14:00"}}

> Finished chain.


' It is currently 25.8°C in Singapore with a windspeed of 10.5 km/h and a winddirection of 16°. The weathercode is 3, which indicates that it is currently raining in Singapore.'